Połączenie z google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Instaowanie rapid

In [2]:
import os, sys, io
import subprocess
from pathlib import Path

# Install RAPIDS -- we're doing this in one file, for now, due to ease of use
try:
  import pynvml
except:
  output = subprocess.Popen(["pip install pynvml"], shell=True, stderr=subprocess.STDOUT,
      stdout=subprocess.PIPE)
  for line in io.TextIOWrapper(output.stdout, encoding="utf-8"):
    if(line == ""):
      break
    else:
      print(line.rstrip())
  import pynvml
try:
  pynvml.nvmlInit()
except:
  raise Exception("""
                  Unfortunately you're in a Colab instance that doesn't have a GPU.

                  Please make sure you've configured Colab to request a GPU Instance Type.

                  Go to 'Runtime -> Change Runtime Type --> under the Hardware Accelerator, select GPU', then try again."""
  )
gpu_name = pynvml.nvmlDeviceGetName(pynvml.nvmlDeviceGetHandleByIndex(0))
rapids_version = "24.4.*"

if ('P' not in gpu_name):
  print('***********************************************************************')
  print('Woo! Your instance has a '+ str(gpu_name)+' GPU!')
  print(f'We will install the latest stable RAPIDS via pip {rapids_version}!  Please stand by, should be quick...')
  print('***********************************************************************')
  print()
else:
  print('***********************************************************************')
  print('Hey! Your instance has a Pascal GPU, a '+ str(gpu_name)+'!')
  print('We will install a compatible RAPIDS via pip (23.12)!  Please stand by, should be quick...')
  print('***********************************************************************')
  print()
  rapids_version = "23.12.*"


output = subprocess.Popen([f"pip install cudf-cu12=={rapids_version} cuml-cu12=={rapids_version} cugraph-cu12=={rapids_version} cuspatial-cu12=={rapids_version} cuproj-cu12=={rapids_version} cuxfilter-cu12=={rapids_version} cucim-cu12=={rapids_version} pylibraft-cu12=={rapids_version} raft-dask-cu12=={rapids_version} aiohttp --extra-index-url=https://pypi.nvidia.com"], shell=True, stderr=subprocess.STDOUT,
    stdout=subprocess.PIPE)
for line in io.TextIOWrapper(output.stdout, encoding="utf-8"):
  if(line == ""):
    break
  else:
    print(line.rstrip())
output = subprocess.Popen(["rm -rf /usr/local/lib/python3.10/dist-packages/cupy*"], shell=True, stderr=subprocess.STDOUT,
    stdout=subprocess.PIPE)
for line in io.TextIOWrapper(output.stdout, encoding="utf-8"):
  if(line == ""):
    break
  else:
    print(line.rstrip())
output = subprocess.Popen(["pip install cupy-cuda12x"], shell=True, stderr=subprocess.STDOUT,
    stdout=subprocess.PIPE)
for line in io.TextIOWrapper(output.stdout, encoding="utf-8"):
  if(line == ""):
    break
  else:
    print(line.rstrip())
print("""
        ***********************************************************************
        The pip install of RAPIDS is complete.

        Please do not run any further installation from the conda based installation methods, as they may cause issues!

        Please ensure that you're pulling from the git repo to remain updated with the latest working install scripts.

        Troubleshooting:
            - If there is an installation failure, please check back on RAPIDSAI owned templates/notebooks to see how to update your personal files.
            - If an installation failure persists when using the latest script, please make an issue on https://github.com/rapidsai-community/rapidsai-csp-utils
        ***********************************************************************
        """
      )

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 1.8 MB/s eta 0:00:00
***********************************************************************
Woo! Your instance has a Tesla T4 GPU!
We will install the latest stable RAPIDS via pip 24.4.*!  Please stand by, should be quick...
***********************************************************************

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 GB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 GB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.8/137.8 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.9/920.9 kB 65.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 108.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.0/823.0 MB 1.9 MB/s eta 0:00:00
     ━━━

Niezbędne importy

In [4]:
import cuml
from sklearn.cluster import Birch
from cuml.metrics.cluster import silhouette_score
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import adjusted_rand_score, davies_bouldin_score
from scipy.spatial.distance import cdist

Wczytanie danych

In [5]:
embeddings_path = '/content/drive/My Drive/data_sety/normalized_word2vec_embeddings.csv'
data_embeddings = pd.read_csv(embeddings_path)

original_data_path = '/content/drive/My Drive/data_sety/ready_data_set.csv'
original_data = pd.read_csv(original_data_path)
labels_true = original_data['label']


Funkcja licząca indeks dunn

In [6]:
def dunn_index(X, labels):
    distances = cdist(X, X, 'euclidean')
    unique_clusters = np.unique(labels)
    inter_cluster_distances = []
    intra_cluster_diameters = []
    for i in unique_clusters:
        for j in unique_clusters:
            if i != j:
                inter_cluster_distances.append(np.min(distances[labels == i][:, labels == j]))
        intra_cluster_diameters.append(np.max(distances[labels == i][:, labels == i]))
    return np.min(inter_cluster_distances) / np.max(intra_cluster_diameters)


Przygotowanie df do zbierania wyników i zklastrowanych danych

In [7]:
results_df = pd.DataFrame()
clustered_data = pd.DataFrame()
results_list = []

In [8]:
silhouette_scores = []
db_scores = []
ari_scores = []
dunn_scores = []

Klasteryzacja z wykorzystaniem 5-krotnej walidacji krzyżowej

In [9]:
data_embeddings_array = data_embeddings.to_numpy()


kf = KFold(n_splits=5, shuffle=True, random_state=42)

fold_number = 0
for train_index, test_index in kf.split(data_embeddings_array):
    fold_number += 1
    X_train, X_test = data_embeddings.iloc[train_index], data_embeddings.iloc[test_index]
    y_train, y_test = labels_true[train_index], labels_true[test_index]

    # Klasteryzacja za pomocą Birch
    birch = Birch(n_clusters=2)
    labels_pred = birch.fit_predict(X_test)

    # Obliczanie metryk
    silhouette = silhouette_score(X_test, labels_pred)
    db_index = davies_bouldin_score(X_test, labels_pred)
    ari = adjusted_rand_score(y_test, labels_pred)
    dunn = dunn_index(X_test, labels_pred)

    # Dodanie wyników do listy wyników w celu obliczenia średniej na koniec
    silhouette_scores.append(silhouette)
    db_scores.append(db_index)
    ari_scores.append(ari)
    dunn_scores.append(dunn)

    results_list.append({
        'Fold': fold_number,
        'Silhouette Score': silhouette,
        'ARI Score': ari,
        'Dunn Index': dunn
    })


    #Zapisywanie zklasteryzowanych danych
    temp_df = pd.DataFrame(X_test)
    temp_df['True Label'] = y_test.values
    temp_df['Predicted Cluster'] = labels_pred
    temp_df['Fold'] = fold_number
    clustered_data = pd.concat([clustered_data, temp_df], ignore_index=True)

    # Wyświetlenie wyników częściowych dla danego etapu
    print(f"Fold {fold_number}: Silhouette Score: {silhouette}, DB Score: {db_index}, ARI Score: {ari}, Dunn Index: {dunn}")

Fold 1: Silhouette Score: 0.2826421373719591, DB Score: 1.9199765405210016, ARI Score: 0.025315056837692203, Dunn Index: 0.10439054446909937
Fold 2: Silhouette Score: 0.27125857136440323, DB Score: 2.0383507721295517, ARI Score: 0.018539708971626174, Dunn Index: 0.04215761111596645
Fold 3: Silhouette Score: 0.12340588037724125, DB Score: 3.5727458683663555, ARI Score: 0.07318136306836798, Dunn Index: 0.01003398525939847
Fold 4: Silhouette Score: 0.27238406766487994, DB Score: 2.01230299577814, ARI Score: 0.03212675864415185, Dunn Index: 0.07304956034243532
Fold 5: Silhouette Score: 0.25473641222488747, DB Score: 1.8753669115938758, ARI Score: 0.019510994019045602, Dunn Index: 0.08105626291417975


Wyświeylenie podsumowania

In [10]:
average_silhouette = np.mean(silhouette_scores)
average_db = np.mean(db_scores)
average_ari = np.mean(ari_scores)
average_dunn = np.mean(dunn_scores)

print("Average Silhouette Score:", average_silhouette)
print("Average Davies-Bouldin Score:", average_db)
print("Average Adjusted Rand Index:", average_ari)
print("Average Dunn Index:", average_dunn)

Average Silhouette Score: 0.2408854138006742
Average Davies-Bouldin Score: 2.283748617677785
Average Adjusted Rand Index: 0.03373477630817676
Average Dunn Index: 0.06213759282021587


Zapisanie wyników

In [11]:
results_file_path = '/content/drive/My Drive/data_sety/word2vec_birch_kfold_clustering_metrics.csv'
clustered_data_file_path = '/content/drive/My Drive/data_sety/word2vec_birch_kfold_clustered_data.csv'

results_df = pd.DataFrame(results_list)
results_df.to_csv(results_file_path, index=False)
clustered_data.to_csv(clustered_data_file_path, index=False)